# ACO - Vehicle Routing Problem with Capacity and Time Windows

In [1]:
from math import dist
from tqdm import tqdm
import copy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns

## Prepare data and graph, visualization of data points

In [2]:
class Graph:
    def __init__(self, x, y, n_depots) -> None:
        self.n = n = len(x)
        self.x = np.array(x, dtype=float)
        self.y = np.array(y, dtype=float)
        self.edges = np.zeros(shape=(n,n), dtype=float)

        for i in range(self.n):
            for j in range(self.n):
                self.edges[i,j] = dist([x[i], y[i]], [x[j], y[j]])

        xf = 10000
        # high distance from one point to itself
        for i in range(self.n):
            self.edges[i,i] = xf
            
        # high distance from one depot to another depot
        for i in range(n_depots):
            for j in range(n_depots):
                self.edges[i,j] = xf

In [3]:
def get_location_data():
    with open('rcdata/r103.txt', 'r', encoding = 'utf-8') as f:
        [f.readline() for i in range(4)]
        n_locations = 100
        max_vehicles, max_capacity = [int(x) for x in f.readline().split()]
        data = []
        [f.readline() for i in range(4)]
        for i in range(n_locations + 1):
            data.append([float(x) for x in f.readline().split()[1:]])

    # uncomment to see plot
    # data = np.array(data)
    # dataT = data.T
    # a = pd.DataFrame({'x':dataT[0], 'y':dataT[1] , 'd':dataT[2]})
    # sns.scatterplot(data=a, x='x', y='y', size='d', palette="deep")
    return data, n_locations, max_vehicles, max_capacity

In [4]:
def insert_duplicate_depots(depots_to_add):
    data, n_locations, max_vehicles, max_capacity = get_location_data()
    data = np.append(np.repeat([data[0]], depots_to_add, axis=0), data, axis=0)
    return data, n_locations, max_vehicles, max_capacity

In [5]:
def create_graph(n_vehicles):
    n_depots = n_vehicles + 1 # IMPORTANT
    data, n_locations, max_vehicles, max_capacity = insert_duplicate_depots(n_depots - 1)
    dataT = data.T
    graph = Graph(dataT[0], dataT[1], n_depots)
    return data, graph, n_locations, n_depots, max_vehicles, max_capacity 

In [6]:
n_vehicles = 10
data, graph, n_locations, n_depots, max_vehicles, max_capacity = create_graph(n_vehicles)

In [7]:
print(data.shape, graph.edges.shape, n_locations, n_depots, max_vehicles, max_capacity)
data[:20]

(111, 6) (111, 111) 100 11 25 200


array([[ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 35.,  35.,   0.,   0., 230.,   0.],
       [ 41.,  49.,  10.,   0., 204.,  10.],
       [ 35.,  17.,   7.,   0., 202.,  10.],
       [ 55.,  45.,  13.,   0., 197.,  10.],
       [ 55.,  20.,  19., 149., 159.,  10.],
       [ 15.,  30.,  26.,   0., 199.,  10.],
       [ 25.,  30.,   3.,  99., 109.,  10.],
       [ 20.,  50.,   5.,   0., 198.,  10.],
       [ 10.,  43.,   9.,  95., 105.,  10.],
       [ 55.,  60.,  16.,  97., 107.,  10.]])

In [8]:
assert(data.shape[0] == n_locations + n_depots and data.shape[1] == 6)
assert(graph.edges.shape[0] == n_locations + n_depots and graph.edges.shape[1] == graph.edges.shape[0])

graph.edges

array([[1.00000000e+04, 1.00000000e+04, 1.00000000e+04, ...,
        2.12602916e+01, 1.74928557e+01, 2.40416306e+01],
       [1.00000000e+04, 1.00000000e+04, 1.00000000e+04, ...,
        2.12602916e+01, 1.74928557e+01, 2.40416306e+01],
       [1.00000000e+04, 1.00000000e+04, 1.00000000e+04, ...,
        2.12602916e+01, 1.74928557e+01, 2.40416306e+01],
       ...,
       [2.12602916e+01, 2.12602916e+01, 2.12602916e+01, ...,
        1.00000000e+04, 5.09901951e+00, 3.16227766e+00],
       [1.74928557e+01, 1.74928557e+01, 1.74928557e+01, ...,
        5.09901951e+00, 1.00000000e+04, 8.24621125e+00],
       [2.40416306e+01, 2.40416306e+01, 2.40416306e+01, ...,
        3.16227766e+00, 8.24621125e+00, 1.00000000e+04]])

## Objective function

In [9]:
def maco_objective(tour, n_locations, n_depots, data, graph, max_capacity):
    
    curr_time, curr_dist, curr_load = 0, 0, 0

    for i in range(len(tour)-1):

        ni, nj = tour[i], tour[i+1]
        tij = dist([data[ni,0], data[ni,1]], [data[nj,0], data[nj,1]])
        curr_dist += tij

        # not necessary, just checking correctness
        if nj < n_depots:   # destination location is a depot
            curr_time, curr_load = 0, 0

        else:               # destination location is a client location
            curr_load += data[nj,2]
            curr_time = max(curr_time + tij, data[nj,3])
            assert(data[nj,3] <= curr_time <= data[nj,4])
            assert(curr_load <= max_capacity)
            curr_time += data[nj,5]
        
    assert(tour[0] < n_depots and tour[-1] < n_depots)
    
    return curr_dist

## Ant Definition

In [10]:
class Ant:
    def __init__(self, tour, n_locations, n_depots, data, graph, max_capacity) -> None:
        self.tour = tour
        self.cost = maco_objective(tour, n_locations, n_depots, data, graph, max_capacity)
        self.veh = sum([x < n_depots for x in tour]) - 1 # CHECK LATER
        # clients visited
        self.visi = len(set(tour)) - (self.veh + 1) # CHECK LATER

    def __repr__(self) -> str:
        return f'Ant(tour={self.tour}, o={self.cost}, veh={self.veh}, visi={self.visi})'

    def __lt__(self, other):
        return self.cost < other.cost

## Calculating feasible solution

In [11]:
def feasible_initial_solution(n_locations, n_depots, graph, data, max_capacity):
    tour = [0]
    vis = np.zeros(n_locations + n_depots)
    vis[0] = 1
    load, curr_time = 0, 0
    next_depot = 1

    while np.sum(vis[-n_locations:]) < n_locations: # while all client locations are yet to be visited
        last = tour[-1]
        mins = np.argsort(graph.edges[last])
        # print(graph.edges[last])
        # print(mins)
        # print(tour)
        for x in mins:
            # considering only unvisited client locations
            if (x >= n_depots and not vis[x] and curr_time + graph.edges[last,x] <= data[x,4]
                and load + data[x, 2] <= max_capacity):
                curr_time = max(curr_time + graph.edges[last][x], data[x, 3])
                curr_time += data[x,5]
                load += data[x, 2]
                tour.append(x)
                vis[x] = 1
                break
        else:
            # if no valid unvisited client locations, return to depot and start new cycle
            curr_time = 0
            load = 0
            tour.append(next_depot)
            vis[next_depot] = 1
            next_depot += 1 
    if tour[-1] >= n_depots:
        tour.append(next_depot)
    sol = Ant(tour, n_locations, n_depots, data, graph, max_capacity)
    return sol

n_vehicles = 30
data, graph, n_locations, n_depots, max_vehicles, max_capacity = create_graph(n_vehicles)
psi_fis = feasible_initial_solution(n_locations, n_depots, graph, data, max_capacity)
print(psi_fis)

Ant(tour=[0, 83, 88, 1, 57, 58, 56, 42, 110, 2, 119, 36, 124, 125, 127, 89, 123, 126, 3, 70, 51, 103, 102, 104, 105, 32, 4, 43, 67, 128, 5, 82, 48, 90, 113, 114, 35, 91, 115, 121, 130, 6, 99, 31, 80, 107, 33, 7, 106, 98, 59, 54, 8, 61, 118, 37, 112, 78, 77, 49, 93, 9, 129, 46, 116, 47, 10, 122, 117, 87, 73, 44, 11, 100, 12, 84, 85, 55, 13, 63, 111, 39, 81, 50, 62, 120, 14, 34, 15, 60, 40, 101, 16, 72, 45, 71, 86, 17, 92, 41, 79, 76, 18, 109, 64, 65, 19, 38, 20, 52, 21, 75, 74, 68, 22, 108, 96, 23, 69, 53, 97, 24, 66, 94, 25, 95, 26], o=1966.5661118342036, veh=26, visi=100)


## ACO Vehicle function

In [12]:
def infeasible_solution(n_locations, n_depots, n_vehicles, graph, data, max_capacity):
    tour = [0]
    vis = np.zeros(n_locations + n_depots)
    vis[0] = 1
    load, curr_time = 0, 0
    next_depot = 1

    while np.sum(vis[-n_locations:]) < n_locations: # while all client locations are yet to be visited
        last = tour[-1]
        mins = np.argsort(graph.edges[last])
        
        for x in mins:
            # considering only unvisited client locations
            if (x >= n_depots and not vis[x] and curr_time + graph.edges[last,x] <= data[x,4]
                and load + data[x, 2] <= max_capacity):
                curr_time = max(curr_time + graph.edges[last][x], data[x, 3])
                curr_time += data[x,5]
                load += data[x, 2]
                tour.append(x)
                vis[x] = 1
                break
        else:
            # if no valid unvisited client locations, return to depot and start new cycle
            curr_time = 0
            load = 0
            tour.append(next_depot)
            vis[next_depot] = 1
            next_depot += 1 
            if next_depot == n_depots:
                break
    if tour[-1] >= n_depots:
        tour.append(next_depot)
    sol = Ant(tour, n_locations, n_depots, data, graph, max_capacity)
    return sol

n_vehicles = 10
data, graph, n_locations, n_depots, max_vehicles, max_capacity = create_graph(n_vehicles)
psi_is = infeasible_solution(n_locations, n_depots, n_vehicles, graph, data, max_capacity)
print(psi_is)

Ant(tour=[0, 63, 68, 1, 37, 38, 36, 50, 31, 83, 82, 84, 85, 12, 2, 99, 16, 104, 105, 107, 69, 103, 106, 3, 23, 47, 108, 4, 62, 28, 70, 93, 94, 15, 71, 26, 101, 110, 5, 79, 11, 60, 87, 13, 6, 22, 90, 7, 86, 78, 39, 34, 8, 41, 98, 17, 92, 58, 57, 29, 73, 9, 109, 95, 24, 53, 67, 10], o=632.5893948187688, veh=10, visi=57)


In [13]:
def valid_tour(tour, n_locations, n_depots, data, graph, max_capacity):
    
    curr_time, curr_dist, curr_load = 0, 0, 0

    for i in range(len(tour)-1):

        ni, nj = tour[i], tour[i+1]
        tij = dist([data[ni,0], data[ni,1]], [data[nj,0], data[nj,1]])
        curr_dist += tij

        # not necessary, just checking correctness
        if nj < n_depots:   # destination location is a depot
            curr_time, curr_load = 0, 0

        else:               # destination location is a client location
            curr_load += data[nj,2]
            curr_time = max(curr_time + tij, data[nj,3])
            if not (data[nj,3] <= curr_time <= data[nj,4]):
                return False
            if not(curr_load <= max_capacity):
                return False
            curr_time += data[nj,5]
        
    if not (tour[0] < n_depots and tour[-1] < n_depots):
        return False    
        
    return True

In [14]:
def new_active_ant(local, incom, n_locations, n_depots, n_vehicles, data, graph, tau, tau0, beta, max_capacity, rho):

    # print(data.shape, graph.edges.shape)
    inc = incom.copy()

    tour = [np.random.choice(n_depots)]
    curr_time, curr_load = 0, 0

    vis = np.zeros(n_locations + n_depots)
    vis[tour[0]] = 1
    eta = np.full(n_locations + n_depots, 0.00001)

    # first insertion of nodes by ACO
    feasible_node = True
    while feasible_node:

        feasible_node = False
        last = tour[-1]
        for x in range(n_locations + n_depots):
            if (not vis[x] and curr_time + graph.edges[last,x] <= data[x,4]
                    and curr_load + data[x, 2] <= max_capacity):
                feasible_node = True
                deliv_time = max(curr_time + graph.edges[last,x], data[x, 3])
                delta_time = deliv_time - curr_time
                distance = delta_time * (data[x,4] - curr_time)
                # print(distance)
                distance = max(1, distance - inc[x])
                eta[x] = 1 / distance
            else:
                eta[x] = 0

        if not feasible_node:
            break

        probs = (tau[last]) * (eta ** beta)
        probs = probs / np.sum(probs)
        
        next = np.random.choice(n_locations + n_depots, p=probs)
        tour.append(next)
        vis[next] = 1

        curr_time = max(curr_time + graph.edges[last][x], data[x, 3])
        curr_time += data[x,5]

        curr_load += data[x, 2]
        # print('A', tour)

        tau[last][next] = (1 - rho) * tau[last][next] + rho * tau0
    
    # print('B', tour)

    # there still may remain some non inserted nodes
    for x in range(n_locations + n_depots):
        if not vis[x]:
            for g in range(1, len(tour) + 1):
                tc = tour.copy()
                tc.insert(g, x)
                if valid_tour(tc, n_locations, n_depots, data, graph, max_capacity):
                    tour = tc
                    vis[x] = 1
                    break
    
    sol = Ant(tour, n_locations, n_depots, data, graph, max_capacity)
    return sol

In [15]:
# n_vehicles = 10
# data, graph, n_locations, n_depots, max_vehicles, max_capacity = create_graph(n_vehicles)
# incom = np.zeros(n_locations + n_depots)
# tau0 = 10/np.sum(graph.edges)
# tau = np.full(graph.edges.shape, tau0)
# beta = 1
# rho = 0.25
# local = False
# new_active_ant(local, incom, n_locations, n_depots, n_vehicles, data, graph, tau, tau0, beta, max_capacity, rho)

In [16]:
# def aco_vehicle(n_vehicles, n_ants, psi_gb, beta, rho):
def aco_vehicle(n_vehicles, n_ants, beta, rho):
    global psi_gb

    n_depots = n_vehicles + 1
    data, graph, n_locations, n_depots, max_vehicles, max_capacity = create_graph(n_vehicles)
    tau0 = 10/np.sum(graph.edges)
    tau = np.full(graph.edges.shape, tau0)

    # initial solution, may be infeasible
    psi_vei = infeasible_solution(n_locations, n_depots, n_vehicles, graph, data, max_capacity)
    # print(psi_vei)

    incom = np.zeros(n_locations + n_depots)
    for i in tqdm(range(10)):
        ants = [None for i in range(n_ants)]

        for i in range(n_ants):
            local = False
            ants[i] = new_active_ant(local, incom, n_locations, n_depots, 
                                    n_vehicles, data, graph, tau, tau0, 
                                    beta, max_capacity, rho)
            for x in ants[i].tour:
                incom[x] += 1
 
        for i in range(n_ants):
            if ants[i].visi > psi_vei.visi:
                psi_vei = copy.deepcopy(ants[i])
                incom = np.zeros(n_locations + n_depots)
                if ants[i].visi == n_locations:
                    psi_gb = copy.deepcopy(ants[i])
            
        # updating tau values
        tau = (1-rho) * tau
        for j in range(len(psi_vei.tour) - 1):
            cur, nxt = psi_vei.tour[j:j+2]
            tau[cur][nxt] += 1/psi_vei.cost
            tau[nxt][cur] += 1/psi_vei.cost
        for j in range(len(psi_gb.tour) - 1):
            cur, nxt = psi_gb.tour[j:j+2]
            if cur < tau.shape[0] and nxt < tau.shape[0]:
                tau[cur][nxt] += 1/psi_gb.cost
                tau[nxt][cur] += 1/psi_gb.cost

    return psi_vei

In [17]:
# n_vehicles = 17
# beta = 1
# rho = 0.25
# n_ants = 20

# aco_vehicle(n_vehicles, n_ants, psi_fis, beta, rho)

## ACO Time function

In [18]:
# def aco_time(n_vehicles, n_ants, psi_gb, beta, rho):
def aco_time(n_vehicles, n_ants, beta, rho):
    global psi_gb

    # psi_gb.cost = 10000
    n_depots = n_vehicles + 1
    data, graph, n_locations, n_depots, max_vehicles, max_capacity = create_graph(n_vehicles)
    tau0 = 10/np.sum(graph.edges)
    tau = np.full(graph.edges.shape, tau0)

    incom = np.zeros(n_locations + n_depots)
    for i in tqdm(range(10)):
        ants = [None for i in range(n_ants)]

        for i in range(n_ants):
            local = False
            ants[i] = new_active_ant(local, incom, n_locations, n_depots, 
                                    n_vehicles, data, graph, tau, tau0, 
                                    beta, max_capacity, rho)
 
        for i in range(n_ants):
            # print(len(ants[i].tour), ants[i].cost)
            if len(ants[i].tour) == n_locations + n_depots and ants[i].cost < psi_gb.cost:
                psi_gb = ants[i]
            
        # updating tau values
        tau = (1-rho) * tau
        # print(psi_gb)
        for j in range(len(psi_gb.tour) - 1):
            cur, nxt = psi_gb.tour[j:j+2]
            if cur < tau.shape[0] and nxt < tau.shape[0]:
                tau[cur][nxt] += 1/psi_gb.cost
                tau[nxt][cur] += 1/psi_gb.cost

    return psi_gb

In [19]:
# n_vehicles = 19
# beta = 1
# rho = 0.25
# n_ants = 20

# aco_time(n_vehicles, n_ants, psi_fis, beta, rho)

## Higher level multiple ACO function

In [20]:
def maco_vrptw(n_ants, beta, rho):
    
    veh = psi_gb.veh
    for i in range(30):

        for i in range(10):
            psi1 = aco_vehicle(veh - 1, n_ants, beta, rho)
            print(psi1)
            if psi1.veh <= veh and psi1.visi == 100:
                veh = psi1.veh
                break
            psi2 = aco_time(veh, n_ants, beta, rho)
            print(psi2)
        
    return psi_gb

In [21]:
n_vehicles = 30
data, graph, n_locations, n_depots, max_vehicles, max_capacity = create_graph(n_vehicles)
psi_gb = feasible_initial_solution(n_locations, n_depots, graph, data, max_capacity)

n_ants = 10
beta = 1
rho = 0.1
maco_vrptw(n_ants, beta, rho)

 20%|██        | 2/10 [00:12<00:50,  6.34s/it]


AssertionError: 